In [12]:
import requests
import os
import pandas as pd
from bs4 import BeautifulSoup
import http.cookies
import re
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from time import sleep
from multiprocessing.dummy import Pool as ThreadPool

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36 Edg/89.0.774.63',
          }

In [3]:
prod_urls = []
for page_num in range(1, 3):
    url = f'https://www.citiwide-online.com/categories/orient-classic-series?limit=72&page={page_num}'
    site = requests.get(url)
    soup = BeautifulSoup(site.content, 'lxml')
    prod_urls += ["https://www.citiwide-online.com"+x.find('a')['href'] for x in soup.findAll('product-item')]

In [49]:
name_array, original_price_array, discount_price_array = [], [], []
img_urls_array, description_html_array, description_text_array = [], [], []
prod_urls_array = []

In [50]:
def scrapify(url):
    site = requests.get(url, headers=headers)
    soup = BeautifulSoup(site.content)
    title = soup.find('div', {'class':'title global-primary dark-primary'}).getText('', True)
    orignal_price = soup.find('div', {'class':'global-primary dark-primary price-regular price js-price price-crossed'}).getText('', True)
    discount_price = soup.find('div', {'class':'price-sale price js-price'}).getText('', True)
    description_text = soup.find('div', {'class':'global-secondary dark-secondary description-container'}).getText('', True)
    description_html = soup.find('div', {'class':'global-secondary dark-secondary description-container'})
    img_urls = []
    for temp in str(soup).split('detail_image_url')[1:]:
        img_urls.append(temp.split('?\\')[0][5:])
        
    
    name_array.append(title)
    original_price_array.append(orignal_price)
    discount_price_array.append(discount_price)
    description_text_array.append(description_text)
    description_html_array.append(description_html)
    img_urls_array.append(img_urls)
    prod_urls_array.append(url)

In [51]:
# for product_url in prod_urls[:5]:
#     scrapify(product_url, url)

In [52]:
pool = ThreadPool(20)
_ = pool.map(scrapify, prod_urls[:])

In [82]:
del df

In [89]:
df = pd.DataFrame(columns=['Handle', 'Title', 'Description(HTML)', 'Description(text)','OriginalPrice',
                           'DiscountedPrice', 'ProductURL', 'Image Src', 'Image Position'
                            ])

In [90]:
for index, imgs in enumerate(img_urls_array):
    for i, src in enumerate(imgs):
        if i== 0:
            df = df.append({
                'Handle': name_array[index],
                'Title': name_array[index],
                'Description(HTML)': description_html_array[index],
                'Description(text)': description_text_array[index],
                'OriginalPrice': original_price_array[index],
                'DiscountedPrice': discount_price_array[index],
                'ProductURL': prod_urls_array[index],
                'Image Src': src,
                'Image Position': i+1
            }, ignore_index = True)
        else:
            df = df.append({
                'Handle': name_array[index],
                'Title': '',
                'Description(HTML)': '',
                'Description(text)': '',
                'OriginalPrice':  '',
                'DiscountedPrice': '',
                'ProductURL': '',
                'Image Src': src,
                'Image Position': i+1
            }, ignore_index = True)

In [92]:
df.head(10)

,Handle,Title,Description(HTML),Description(text),OriginalPrice,DiscountedPrice,ProductURL,Image Src,Image Position
0,ORIENT 東方萬年曆機械手錶 SEU07005BX/FEU07005BX,ORIENT 東方萬年曆機械手錶 SEU07005BX/FEU07005BX,"<div class=""global-secondary dark-secondary de...",不銹鋼錶殼不銹鋼手鍊自動機芯礦物水晶模擬顯示黑色錶盤日期和日期顯示擰回螺絲冠熒光箭頭和標籤1...,"HK$2,550.00","HK$1,150.00",https://www.citiwide-online.com/products/5ec39...,https://shoplineimg.com/5d02035403a5360001dbbe...,1
1,ORIENT 東方萬年曆機械手錶 SEU07005BX/FEU07005BX,,,,,,,https://shoplineimg.com/5d02035403a5360001dbbe...,2
2,ORIENT東方之星 RETROGRADE系列 經典星期逆跳機械錶 RK-DE0301L,ORIENT東方之星 RETROGRADE系列 經典星期逆跳機械錶 RK-DE0301L,"<div class=""global-secondary dark-secondary de...",產品規格品牌：東方系列：星型號：RK-DE0301L機芯：機械自動機芯編號：40A52驅動系...,"HK$7,840.00","HK$5,880.00",https://www.citiwide-online.com/products/5feac...,https://shoplineimg.com/5d02035403a5360001dbbe...,1
3,ORIENT東方之星 RETROGRADE系列 經典星期逆跳機械錶 RK-DE0301L,,,,,,,https://shoplineimg.com/5d02035403a5360001dbbe...,2
4,ORIENT東方之星 RETROGRADE系列 經典星期逆跳機械錶 RK-DE0301L,,,,,,,https://shoplineimg.com/5d02035403a5360001dbbe...,3
5,東方之星 - Orient Star 潛水系列 200m 防水男士手錶 RK-AU0303B,東方之星 - Orient Star 潛水系列 200m 防水男士手錶 RK-AU0303B,"<div class=""global-secondary dark-secondary de...",產品規格品牌：東方系列：東方之星性別：男士機芯：自動口徑：F6N47錶殼材料：不銹鋼錶殼形狀...,"HK$6,095.00","HK$3,950.00",https://www.citiwide-online.com/products/605c0...,https://shoplineimg.com/5d02035403a5360001dbbe...,1
6,東方之星 - Orient Star 潛水系列 200m 防水男士手錶 RK-AU0303B,,,,,,,https://shoplineimg.com/5d02035403a5360001dbbe...,2
7,東方之星 - Orient Star 潛水系列 200m 防水男士手錶 RK-AU0303B,,,,,,,https://shoplineimg.com/5d02035403a5360001dbbe...,3
8,Orient 東方200米潛水系列自動機械手錶 RA-AC0K05G00C,Orient 東方200米潛水系列自動機械手錶 RA-AC0K05G00C,"<div class=""global-secondary dark-secondary de...",產品規格機芯：F6722機芯自動上鍊黑客運動動力儲備：約40小時特點：日期，發光標記晶體：藍...,"HK$3,750.00","HK$2,430.00",https://www.citiwide-online.com/products/601b7...,https://shoplineimg.com/5d02035403a5360001dbbe...,1
9,Orient 東方200米潛水系列自動機械手錶 RA-AC0K05G00C,,,,,,,https://shoplineimg.com/5d02035403a5360001dbbe...,2


In [74]:
df.drop(['DiscountedPriceProductURL'], axis=1, inplace=True)

In [94]:
df.to_csv('citywide - orient.csv', index=False)

In [95]:
df.to_excel('citywide - orient.xlsx', index=False)